In [1]:
import os 

In [2]:
storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]

In [3]:
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("Read_csv_from_azure_blob") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g") \
    .set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", access_key) 


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

25/06/11 04:26:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
container = "datalake"
endpoint = f"abfss://{container}@{storage_account}.dfs.core.windows.net"

In [6]:
folder = "user"

In [7]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("pathGlobFilter", "*.csv") \
    .load(f"{endpoint}/{folder}")
df.show()


+---+-------------------+
| id|               name|
+---+-------------------+
|  6|marselino ferdinand|
|  7|          jay idzes|
|  8|         ole romeny|
|  4|  cristiano ronaldo|
|  5|       lionel messi|
+---+-------------------+

